In [1]:
import os
import numpy as np
import pandas as pd

from sklearn import metrics # Metrics for classification
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score
from sklearn.model_selection import KFold
from sklearn import metrics
from sklearn.neural_network import MLPClassifier

from statsmodels.stats import contingency_tables # Contringency tables
import torch # PyTorch
from torch import nn # Modules and layers
from torch.optim import SGD
from torch.utils.data import Dataset # PyTorch dataset
from torch.utils.data import DataLoader # PyTorch Dataloader

In [2]:
from sklearn.neural_network import  MLPClassifier

## Data Loading

In [3]:
# Read csv dataset
df = pd.read_csv('data_assignment_1.csv')
df.describe()

# input
feature_cols = ['Frontal_Sup', 'Frontal_Inf', 'Cingulum_Ant', 'Cingulum_Post', 'Parietal_Sup', 'Parietal_Inf', 'Occipital_Sup', 'Occipital_Inf', 'Temporal_Sup', 'Temporal_Inf']
# feature_cols = ['Cingulum_Ant', 'Cingulum_Post']
# output (label)
target_col = ['diagnosis']

## Datasets and Dataloaders


In [4]:
class BrainDataset(Dataset):
    def __init__(self, df, feature_cols, target_col):
        # data loading
        self.X = torch.tensor(df[feature_cols].values, dtype=torch.float32) # inputs: brain regional biomarkers
        self.y = torch.tensor(df[target_col].values, dtype=torch.float32) # output: diagnosis (0 or 1)
        # self.n_samples = len(self.y)

    def __getitem__(self, index):
        # dataset[0]
        return self.X[index], self.y[index]

    def __len__(self):
        # len(data)
        return len(self.y)


brain_data = BrainDataset(df, feature_cols, target_col)
# print(brain_data.y)

Cross-Validation

In [5]:
kf = KFold(n_splits=5, shuffle=True, random_state=1)

batch_size = 25
dataloaders = []

for train_idx, test_idx in kf.split(df):
    train_df_kfold = df.iloc[train_idx]
    test_df_kfold = df.iloc[test_idx]

    train_ds_kfold = BrainDataset(train_df_kfold, feature_cols=feature_cols, target_col=target_col)
    test_ds_kfold = BrainDataset(test_df_kfold, feature_cols=feature_cols, target_col=target_col)

    train_dl_kfold = DataLoader(train_ds_kfold, batch_size=batch_size, shuffle=True)
    test_dl_kfold = DataLoader(test_ds_kfold, batch_size=batch_size, shuffle=False)

    dataloaders.append((train_dl_kfold, test_dl_kfold))

# # --------------------------- TESTPRINT
# # Retrieve the dataloader for one of the folds (e.g., the first fold)
# train_dataloader, _ = dataloaders[3]  # getting the train dataloader for the first fold

# # Retrieve one batch of data
# batch = next(iter(train_dataloader))
# inputs, outputs = batch

# # Print or inspect the inputs and outputs
# print("Input features for one batch:", inputs)
# print("Outputs for one batch:", outputs)


In [6]:

# # divide the 'df' into training and testing
# train_df = df.sample(frac = 0.75)
# test_df = df.drop(train_df.index)
# # prepare the datasets in the appropiate format 'ds'
# train_ds = BrainDataset(train_df, feature_cols=feature_cols, target_col=target_col)
# test_ds = BrainDataset(test_df, feature_cols=feature_cols, target_col=target_col)
# # Create dataloaders
# train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True) # Shuffle the train dataset!
# test_dl = DataLoader(test_ds, batch_size=batch_size, shuffle=False)

## Classifier

In [37]:
input_size = len(feature_cols) # Brain regions biomarkers value
hidden_size = 70 # any number
output_size = 1 # diagnosis (yer or no)

class AdClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(AdClassifier, self).__init__()
        self.linear1 = nn.Linear(input_size, hidden_size)  # First (input) layer
        self.relu = nn.LeakyReLU()                         # nonlinear activation function
        self.linear2 = nn.Linear(hidden_size, output_size) # Second (hidden) layer
        self.sigmoid = nn.Sigmoid()                        # Sigmoid for binary classification
        self.flatten = nn.Flatten()
    
    def forward(self, x):
        out = self.flatten(x)
        out = self.linear1(out)
        out = self.relu(out)
        out = self.linear2(out)
        output = self.sigmoid(out)
        return output
model = AdClassifier(input_size,hidden_size,output_size)
# for param in model.parameters():
#     param.requires_grad = True
for param in model.parameters():
    param.requires_grad = True
    print(param.requires_grad)

True
True
True
True


In [43]:
# loss and optimizer
learning_rate = 0.0001
loss_func = nn.BCELoss()
# optimizer = torch.optim.LBFGS(model.parameters(), lr=learning_rate)
optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate)

## Training Loop(Other Optimizer)

In [44]:
def train_model(model, train_dl, optimizer, loss_func, num_epochs, print_loss_every=100, normalize=False):
    for epoch in range(num_epochs):

        total_loss = 0.0
        num_batches = len(train_dl)
        optimizer.zero_grad()
        for X, y in train_dl:
            
            # Normalize data if flag is set
            if normalize:
                X = normalize_data(X)
            # forward and loss
            y_predicted = model(X)
            
            # print(y_predicted.dtype)

            loss = loss_func(y_predicted, y)
            # backward
            loss.backward()
            #update
            optimizer.step()
            optimizer.zero_grad()
            # accumulate loss for entire dataloader
            total_loss += loss.item()
        
        avg_loss = total_loss / num_batches
        if (epoch+1) % print_loss_every == 0:
            print(f'epoch: {epoch+1}, loss: {avg_loss:.4f}')

            # Print the weights and biases
            for name, param in model.named_parameters():
                if 'weight' in name:
                    print(f'{name} weights: {param.data}')
                if 'bias' in name:
                    print(f'{name} biases: {param.data}')

        # for param in model.parameters():
        #     if param.requires_grad:
        #         if param.grad is not None:
        #             print(param.grad.data.norm())
        #         else:
        #             print("Gradient is None for this parameter.")


NameError: name 'train_dl' is not defined

In [45]:
# Evaluation loop
def eval_model(model, test_dl):
    y_true = []
    y_pred = []
    for X, y in test_dl:
        pred = model(X)
        y_true += list(y.detach().cpu().numpy())
        y_pred += list(pred.argmax(1).detach().cpu().numpy())

    # accuracy = metrics.accuracy_score(targets, predictions)
    # balanced_accuracy = metrics.balanced_accuracy_score(targets, predictions)

    # Calculate metrics
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    auc = roc_auc_score(y_true, y_pred)

    print(f"y_pred:, {y_pred},y_true:, {y_true} Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, AUC: {auc}" )

Evalutaion

In [47]:
# # Assuming you have already defined your model, optimizer, loss function etc.
# # Store metrics over the 5 folds
# num_epochs = 2000
# accuracies = []
# precisions = []
# recalls = []
# aurocs = []

# # Train model on each k-fold
# for train_dl_kfold, test_dl_kfold in dataloaders:
#     # Assuming you have a function 'train_model' for training your model
#     train_model(model, train_dl_kfold, optimizer, loss_func, num_epochs)
    
#     # Evaluate model
#     print("Performance on the train set")
#     eval_model(model, train_dl_kfold)

#     print("Performance on the test set")
#     eval_model(model, test_dl_kfold)

# MLPClassifier

In [66]:
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold

# define classifier type
classifier_type ='mlp'
# classifier_type ='svm'
# classifier_type ='logistic'

# KFold splitter
kf = KFold(n_splits=5, shuffle=True, random_state=1)

data_splits = []

for train_idx, test_idx in kf.split(df):
    train_df_kfold = df.iloc[train_idx]
    test_df_kfold = df.iloc[test_idx]
    data_splits.append((train_df_kfold, test_df_kfold))

# Metrics containers
accuracies_train = []
precisions_train = []
recalls_train = []
aurocs_train = []
accuracies_test = []
precisions_test = []
recalls_test = []
aurocs_test = []


for train_df, test_df in data_splits:
    X_train = train_df[feature_cols].values
    y_train = train_df[target_col].values
    X_test = test_df[feature_cols].values
    y_test = test_df[target_col].values
    
    # Choose classifier based on switch
    if classifier_type == 'mlp':
        clf = MLPClassifier(hidden_layer_sizes = 70, max_iter=2000)
    elif classifier_type == 'svm':
        clf = SVC(kernel='linear',decision_function_shape='ovr',probability=True) # Set probability to True to use roc_auc_score
    elif classifier_type == 'logistic':
        clf = LogisticRegression(max_iter=2000)
    clf.fit(X_train, y_train)

    # # print mlp specs
    # print("Optimizer:", clf.solver)
    # print("Loss Function:", clf.loss)
    # print("Learning Rate:", clf.learning_rate_init)
    # print("Layer Nodes:", clf.hidden_layer_sizes)

    # # print svm specs
    # print("Kernel:", clf.kernel)
    # print("Decision Function Shape:", clf.decision_function_shape)


    # Predict
    y_train_pred = clf.predict(X_train)
    y_test_pred = clf.predict(X_test)

    # Evaluate on train set
    accuracies_train.append(accuracy_score(y_train, y_train_pred))
    precisions_train.append(precision_score(y_train, y_train_pred))
    recalls_train.append(recall_score(y_train, y_train_pred))
    aurocs_train.append(roc_auc_score(y_train, y_train_pred))
    # Evaluate on test set
    accuracies_test.append(accuracy_score(y_test, y_test_pred))
    precisions_test.append(precision_score(y_test, y_test_pred))
    recalls_test.append(recall_score(y_test, y_test_pred))
    aurocs_test.append(roc_auc_score(y_test, y_test_pred))

 # Calculate and report results for each classifier
print(f"Results for {classifier_type.upper()}")
print("Average Accuracy over 5-folds: {:.2f} ± {:.2f}".format(np.mean(accuracies_test), np.std(accuracies_test)))
print("Average Precision over 5-folds: {:.2f} ± {:.2f}".format(np.mean(precisions_test), np.std(precisions_test)))
print("Average Recall over 5-folds: {:.2f} ± {:.2f}".format(np.mean(recalls_test), np.std(recalls_test)))
print("Average AUROC over 5-folds: {:.2f} ± {:.2f}".format(np.mean(aurocs_test), np.std(aurocs_test)))
print("-----------------------------------------------------")


C:\Users\yuzho\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1102: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\yuzho\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1102: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\yuzho\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1102: DataConversionWarning: A column-vector y

Results for MLP
Average Accuracy over 5-folds: 0.64 ± 0.05
Average Precision over 5-folds: 0.67 ± 0.12
Average Recall over 5-folds: 0.61 ± 0.15
Average AUROC over 5-folds: 0.66 ± 0.05
-----------------------------------------------------


C:\Users\yuzho\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (2000) reached and the optimization hasn't converged yet.
  warnings.warn(
